# Virginia Drug Overdose Deaths by County Mapper Project

In [1]:
import sklearn
import sklearn.manifold as manifold
import umap
import kmapper as km
import numpy as np
import matplotlib.pyplot as plt
from kmapper.jupyter import display
import pandas as pd

In [2]:
df = pd.read_csv('/Users/joshturner/Desktop/DrugOverdoseTDAVA/Data/OverdoseDeathsByRegion.csv')
# Dataset: https://data.virginia.gov/dataset/vdh-pud-overdose-deaths-by-year-and-geography/resource/a3a62450-44f6-4db2-b400-318c57674b11

In [6]:
# Convert date column
df['Data Extract Date'] = pd.to_datetime(df['Data Extract Date'], errors='coerce')
# Convert year to integer
df['Overdose Death Year'] = pd.to_numeric(df['Overdose Death Year'], errors='coerce').astype('Int64')
# Convert death count and rate to numeric
df['Overdose Death Count'] = pd.to_numeric(df['Overdose Death Count'], errors='coerce')
df['Overdose Death Rate per 100,000 Residents'] = pd.to_numeric(df['Overdose Death Rate per 100,000 Residents'], errors='coerce')
# Convert text fields to string explicitly (optional but improves clarity)
text_columns = [
    'Overdose Death Drug Class',
    'Overdose Death Geography Level',
    'Overdose Death Geography Name',
    'Overdose Death Health District'
]
df[text_columns] = df[text_columns].astype('string')
# Optionally, inspect types to confirm
print(df.dtypes)

_id                                                   int64
Data Extract Date                            datetime64[ns]
Overdose Death Year                                   Int64
Overdose Death Drug Class                    string[python]
Overdose Death Geography Level               string[python]
Overdose Death FIPS                                   int64
Overdose Death Geography Name                string[python]
Overdose Death Health District               string[python]
Overdose Death Count                                float64
Overdose Death Rate per 100,000 Residents           float64
dtype: object


In [8]:
df.shape

(20702, 10)

# Grabs the longitude and latitude for each city

In [7]:
counties = df["Overdose Death Geography Name"].unique()

In [9]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# Initialize geolocator
geolocator = Nominatim(user_agent="va_county_geocoder")

# Create empty lists to store results
latitudes = []
longitudes = []

# Loop over each county
for county in counties:
    location = None
    try:
        # Add 'Virginia, USA' to narrow it down
        location = geolocator.geocode(f"{county}, Virginia, USA")
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    except Exception as e:
        print(f"Error geocoding {county}: {e}")
        latitudes.append(None)
        longitudes.append(None)
    
    # Be polite: Nominatim recommends 1-second delay
    sleep(1)

# Combine into a DataFrame
df_geo = pd.DataFrame({
    "County": counties,
    "Latitude": latitudes,
    "Longitude": longitudes
})

df_geo.shape

Error geocoding Norfolk City: Service timed out
Error geocoding Fairfax City: Service timed out
Error geocoding Petersburg City: Service timed out


(135, 3)

In [10]:
# Fill in the missing longitude and latitude coordinates (luckily for this dataset there were only two)
missing_coords = {
    "Rockbridge County": (37.7776, -79.4308),
    "Lexington City": (37.7840, -79.4428)
}

# Fill in missing manually
for county, (lat, lon) in missing_coords.items():
    df_geo.loc[df_geo["County"] == county, "Latitude"] = lat
    df_geo.loc[df_geo["County"] == county, "Longitude"] = lon


# Merge both datasets

In [11]:
# Merge based on matching county/city names
df_combined = pd.merge(
    df,
    df_geo,
    left_on="Overdose Death Geography Name",
    right_on="County",
    how="left"  # keep all rows in df, even if there's no match in df_geo
)


In [12]:
df_combined.shape

(20702, 13)

In [14]:
df_combined.isna().sum()

_id                                             0
Data Extract Date                               0
Overdose Death Year                          5896
Overdose Death Drug Class                       0
Overdose Death Geography Level                  0
Overdose Death FIPS                             0
Overdose Death Geography Name                   0
Overdose Death Health District                154
Overdose Death Count                          318
Overdose Death Rate per 100,000 Residents     384
County                                          0
Latitude                                      462
Longitude                                     462
dtype: int64

In [15]:
df_cleaned = df.drop_duplicates()
df_cleaned.shape

(20702, 10)

In [16]:
df_combined.to_csv('/Users/joshturner/Desktop/DrugOverdoseTDAVA/Data/VDHPUDOverdoseDeathsByYearAndGeographyLongLat.csv')


In [47]:
df_combined.columns

Index(['_id', 'Data Extract Date', 'Overdose Death Year',
       'Overdose Death Drug Class', 'Overdose Death Geography Level',
       'Overdose Death FIPS', 'Overdose Death Geography Name',
       'Overdose Death Health District', 'Overdose Death Count',
       'Overdose Death Rate per 100,000 Residents', 'County', 'Latitude',
       'Longitude'],
      dtype='object')

# Prep for Mapper

In [293]:
df.nunique()

_id                                          20702
Data Extract Date                                1
Overdose Death Year                              7
Overdose Death Drug Class                       11
Overdose Death Geography Level                   2
Overdose Death FIPS                            135
Overdose Death Geography Name                  135
Overdose Death Health District                  36
Overdose Death Count                           196
Overdose Death Rate per 100,000 Residents      586
dtype: int64

In [296]:
df[df["Overdose Death Geography Name"] == df["Overdose Death Geography Name"].unique().tolist()[0]]

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents"
0,1,6/2/2025,2021,Any Opioids,Locality,51175,Southampton County,Western Tidewater,2,11.3
110,111,6/2/2025,2019,Cocaine,Locality,51175,Southampton County,Western Tidewater,0,0
142,143,6/2/2025,2023,Fentanyl and Other Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
516,517,6/2/2025,2019,Fentanyl and Other Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
613,614,6/2/2025,2020,Any Opioids,Locality,51175,Southampton County,Western Tidewater,4,22.7
...,...,...,...,...,...,...,...,...,...,...
19712,19713,6/2/2025,2019,Methadone,Locality,51175,Southampton County,Western Tidewater,0,0
19750,19751,6/2/2025,2022,Natural and Semi-Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7
20117,20118,6/2/2025,2025†,Methadone,Locality,51175,Southampton County,Western Tidewater,0,0
20177,20178,6/2/2025,2021,Natural and Semi-Synthetic Opioids,Locality,51175,Southampton County,Western Tidewater,1,5.7


In [321]:
features = ['County','Overdose Death Drug Class','Overdose Death Year','Overdose Death Count', 'Latitude', 'Longitude']

mapper_data = df_combined[features]

In [ ]:
# Keep only digits in the Year column
mapper_data['Overdose Death Year'] = mapper_data['Overdose Death Year'].str.extract('(\d{4})')  # Extracts 4-digit year
mapper_data['Overdose Death Year'] = pd.to_numeric(mapper_data['Overdose Death Year'], errors='coerce')  # Convert to numeric (int or NaN)
# Keep only digits in the Year column
# mapper_data['Overdose Death Count'] = mapper_data['Overdose Death Year'].str.extract('(\d{4})')  # Extracts 4-digit year
mapper_data['Overdose Death Count'] = pd.to_numeric(mapper_data['Overdose Death Count'], errors='coerce')  # Convert to numeric (int or NaN)


In [ ]:
mapper_data.dropna(inplace=True)

In [ ]:
mapper_data.shape

(20384, 6)

In [327]:
df.iloc[[2,10353]]

,_id,Data Extract Date,Overdose Death Year,Overdose Death Drug Class,Overdose Death Geography Level,Overdose Death FIPS,Overdose Death Geography Name,Overdose Death Health District,Overdose Death Count,"Overdose Death Rate per 100,000 Residents"
2,3,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8
10353,10354,6/2/2025,2020,Any Opioids,Locality,51690,Martinsville City,West Piedmont,8,64.8


In [328]:
mapper_data = mapper_data[mapper_data['County'] != 'Virginia']

In [329]:
mapper_data = mapper_data[mapper_data['Overdose Death Year'] == 2020]

In [330]:
mapper_data = mapper_data[mapper_data['Overdose Death Drug Class'] == "Any Opioids"]

In [331]:
mapper_data = mapper_data.drop_duplicates()

In [332]:
print("Total rows:", len(mapper_data))
print("Unique rows:", len(mapper_data.drop_duplicates()))


Total rows: 134
Unique rows: 134


In [333]:
mapper_data[mapper_data.duplicated(keep=False)]


,County,Overdose Death Drug Class,Overdose Death Year,Overdose Death Count,Latitude,Longitude


# Drug Overdoses By Count (Begin)

In [ ]:
import kmapper as km
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import sklearn
import pandas as pd
from sklearn import ensemble
from kmapper.plotlyviz import *
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)

# Step 1: Select and standardize your features
map_features = ['Latitude', 'Longitude','Overdose Death Count']


# Calculate the mean
mean_value = mapper_data['Overdose Death Count'].mean()
# Create a new binary column
mapper_data['Overdose Binary'] = (mapper_data['Overdose Death Count'] > mean_value).astype(int)
# Optional: label as 'low'/'high' instead of 0/1
mapper_data['Overdose Level'] = mapper_data['Overdose Binary'].map({0: 'low', 1: 'high'})

X = np.array(mapper_data[map_features])
y = np.array(mapper_data['Overdose Level'])
# Isolation Forest for lens1
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0],1))



#X_scaled = StandardScaler().fit_transform(X)

# Step 2: Define a lens function (1D)
# lens = mapper_data['Overdose Death Year'].values.reshape(-1, 1)  # <- Optional but useful as a lens

# Optional: Standardize the lens too (depends on whether year differences are meaningful)
# from sklearn.preprocessing import MinMaxScaler
# lens_scaled = MinMaxScaler().fit_transform(lens)

# Step 3: Initialize Mapper object
mapper = km.KeplerMapper(verbose=0)
# l2 norm for lens2
lens2 = mapper.fit_transform(X,projection="l2norm")
lens = np.c_[lens1,lens2]
# Step 4: Fit the data using the lens
scomplex = mapper.map(
    lens,
    # lens_scaled,              # <- this is the lens
    X,                 # <- this is the data
    cover=km.Cover(n_cubes=5, perc_overlap=0.1),
   #clusterer=DBSCAN(eps=0.5, min_samples=3))
   clusterer=sklearn.cluster.KMeans(n_clusters=2,
                                     random_state=3471))

#tooltips = np.array(mapper_data["County"].tolist()) # Format counties)

# Step 5: Visualize
#mapper.visualize(
#    graph,
#    path_html="mapper_overdose_2lens.html",
#    title="TDA Mapper: Overdose Patterns by County and Year",
#    custom_tooltips=tooltips  # Optional, for hover info
#)


In [351]:
global_mean_deaths = mapper_data['Overdose Death Count'].mean()


In [352]:
global_mean_deaths

np.float64(13.992537313432836)

In [353]:
pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]


In [354]:
custom_tooltips = []

for i, row in mapper_data.iterrows():
    county = row["County"]
    death = row["Overdose Death Count"]
    level = "High" if death > global_mean_deaths else "Low"
    
    tooltip_text = (
        f"<b>Death Count:</b> {death}<br>"
        f"<b>Level:</b> {level}<br>"
        f"<b>County:</b> {county}"
    )
    
    custom_tooltips.append(tooltip_text)
custom_tooltips = np.array(custom_tooltips)

In [355]:
color_values = mapper_data['Overdose Death Count'].values  # per data point
#my_colorscale = 'plasma'
import plotly.colors as pc

my_colorscale = pc.sequential.Plasma  # ← this is valid

kmgraph, mapper_summary, colorf_distribution = get_mapper_graph(
    scomplex,
    color_values=color_values,
    color_function_name="Average Overdose Deaths",
    colorscale=pl_brewer,
    custom_tooltips=custom_tooltips  # ← this is key
)


In [357]:
mapper.visualize(
    scomplex,
    path_html="mapper_opioid_overdose_2020.html",
    title="Overdose Deaths By Opioids Mapper 2020",
    color_values=color_values,
    color_function_name="Overdose Death Count",
    colorscale=pl_brewer,
    custom_tooltips=custom_tooltips
)


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Overdose Deaths By Opioids Mapper 2020 | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-

# Drug Overdoses By Count (End)